In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
sys.path.append("../")
import housinginsights
from housinginsights.ingestion import ManifestReader, DataReader
from housinginsights.ingestion import ACSRentCleaner, GenericCleaner

import json


ACS cleaner loaded


In [3]:
#copied from scripts
def load_meta_data(filename='meta.json'):
    """
    Expected meta data format:
        { tablename: {fields:[
            {   "display_name": "Preservation Catalog ID",
                "display_text": "description of what this field is",
                "source_name": "Nlihc_id",
                "sql_name": "nlihc_id",
                "type": "object"
            }
            ]}
        }
    """
    with open(filename) as fh:
        meta = json.load(fh)

    json_is_valid = True
    try:
        for table in meta:
            for field in meta[table]['fields']:
                for key in field:
                    if key not in ('display_name', 'display_text', 'source_name', 'sql_name', 'type'):
                        json_is_valid = False
                        first_json_error = "Location: table: {}, section: {}, attribute: {}".format(table, field, key)
                        raise ValueError("Error found in JSON, check expected format. {}".format(first_json_error))
    except:
        raise ValueError("Error found in JSON, check expected format.")

    return meta


In [4]:
meta = load_meta_data('meta_sample.json')
manifest = ManifestReader('manifest_sample.csv')

In [8]:
from housinginsights.ingestion import CSVWriter

from housinginsights.tools import dbtools
from housinginsights.ingestion import DataSql


#temporary, to test pandas implementation version
from sqlalchemy import create_engine
db_string = dbtools.get_connect_str("local_database")
engine = create_engine(db_string)

#temporary, cursor instead
#cursor = dbtools.get_psycopg2_cursor("local_database")

#actual, use connection
db_conn = dbtools.get_database_connection("local_database")
dbapi_conn = db_conn.connection
cursor = dbapi_conn.cursor()


In [ ]:

for manifest_row in manifest:
    if manifest_row['include_flag'] == 'use': #temp - see load_data for better version
        csv_reader = DataReader(manifest_row=manifest_row)
        csv_writer = CSVWriter(meta = meta, manifest_row = manifest_row)

        sql_writer = DataSql(meta, manifest_row, db_conn)

        #Assign an appropriate testing cleaner
        if manifest_row['destination_table'] == "acs_rent_median_temp":
            cleaner = ACSRentCleaner(meta, manifest_row)
        else:
            cleaner = GenericCleaner(meta,manifest_row)

        for idx, data_row in enumerate(csv_reader):
            clean_data_row = cleaner.clean(data_row, idx)
            if clean_data_row != None:
                csv_writer.write(clean_data_row)
            
        csv_writer.close()
        sql_writer.create_table()
        sql_writer.read_and_write_to_sql()
    
db_conn.close()
print(db_conn)

ready to append


In [7]:
db_conn = dbtools.get_database_connection("local_database")
result = db_conn.execute("SELECT * FROM acs_rent_median_temp")

for r in result:
    print(r)


('1400000US11001000100', '11001000100', 'Census Tract 1, District of Columbia, District of Columbia', 1980, '375')
('1400000US11001000201', '11001000201', 'Census Tract 2.01, District of Columbia, District of Columbia', 0, '**')
('1400000US11001001500', '11001001500', 'Census Tract 15, District of Columbia, District of Columbia', 3500, '***')
('1400000US11001002701', '11001002701', 'Census Tract 27.01, District of Columbia, District of Columbia', 1270, '111')
('1400000US11001004902', '11001004902', 'Census Tract 49.02, District of Columbia, District of Columbia', 1386, '491')
('1400000US11001007504', '11001007504', 'Census Tract 75.04, District of Columbia, District of Columbia', 778, '56')
